In [1]:
# import libraries
import numpy as np
import torch
import torch.nn.functional as F
from tslearn.preprocessing import TimeSeriesScalerMinMax
from torch.utils.data import Dataset


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [2]:
# Load Data 
cluster_id = 6

data_path = '/Users/muneeza/Documents/GitHub/DATA_SMest/'

# Set description variables
sub_feat_names = ['AREA', 'PRECIP' , 'ET', 'SW_END', 'PERC', 'GW_RCHG', 'DA_RCHG', 'REVAP', 'SA_IRR', 'DA_IRR', 'SA_ST', 'DA_ST',
                 'WYLD', 'DAILYCN', 'TMP_AV', 'SOL_TMP', 'SOLAR']

train_id_en = 27*12
test_id_en = 34*12 

data=[]
# Read data for the cluster -  (timeseries, hrus , features)
for i in range(cluster_id):
    data.append(np.load(data_path+'sub_hru_sub_data_clstr_'+str(i)+'.npy')) # (tstep , hrus, features)

# Select id for target variable 
target_id = sub_feat_names.index('SW_END')

In [3]:
# Create an array with data from different clusters 
# shape (hrus, timesteps , features)
num_hrus= [x.shape[1] for x in data]
data_array = np.zeros((456,sum(num_hrus), 17))
id = 0
for i, hrus in enumerate(num_hrus):
    data_array[:,id:id+hrus,:] = data[i]
    id = id+ hrus


In [4]:
# Test Train Split 
X_train = data_array[ 0:train_id_en, : ,  :]
X_test = data_array[ train_id_en: test_id_en , : ,  :]

# Normalize data
X_train = TimeSeriesScalerMinMax(value_range=(-1,1)).fit_transform( np.transpose(X_train,(1,0,2)) )
X_test = TimeSeriesScalerMinMax(value_range=(-1,1)).fit_transform( np.transpose(X_test,(1,0,2)) )
X_train = np.transpose(X_train, (1,0,2) ) # reshape back to original 
X_test = np.transpose(X_test, (1,0,2) ) # reshape back to original 

In [5]:
# Convert to supervised learneing example, target is t+1
seq_len = 12
n_feat = len(sub_feat_names)
n_hrus = X_train.shape[1]
test_len = X_test.shape[0]

Y_train = np.zeros((train_id_en - seq_len , n_hrus, seq_len))
Y_test = np.zeros((test_len - seq_len , n_hrus, seq_len))

for i in range(1 , train_id_en - seq_len):
    Y_train[i, : , :] = X_train[i:i+seq_len  , : , target_id ].T

for i in range(1 , test_len - seq_len):    
    Y_test[i, : , :] = X_test[i:i+seq_len , : , target_id].T
    

X_train = X_train[0:train_id_en-seq_len , : , :  ]
X_test = X_test[0:test_len-seq_len , : , : ]

In [6]:


class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
        self.features = features
        self.target = target
        self.sequence_length = sequence_length
        self.y = torch.tensor(dataframe[target].values).float()
        self.X = torch.tensor(dataframe[features].values).float()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

In [7]:
class LSTM(torch.nn.Module):
    def __init__(self, output_size, n_feat, hidden_dim, n_layers):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = torch.nn.LSTM(n_feat, hidden_dim, n_layers)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden, batch_size):
        lstm_out, hidden = self.lstm(x, hidden)
        out = self.fc(lstm_out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [34]:
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTM(output_size=12 , n_feat= n_feat, hidden_dim=32, n_layers=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()
hist = []
train_dataset = torch.tensor(X_train).double()
model = model.double()
batch_size = X_train.shape[1]
for epoch in tqdm(range(40)):
    cost = torch.tensor([0.0],requires_grad=True)
    hidden = model.init_hidden(batch_size = batch_size)
    y_hat = model(train_dataset, hidden, batch_size=batch_size)
    cost = cost + torch.mean((y_hat[0].data-Y_train)**2)
    cost = cost / (epoch+1)
    hist.append(cost.item())
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()

  0%|          | 0/40 [00:00<?, ?it/s]

torch.Size([312, 9683, 12])


  2%|▎         | 1/40 [00:08<05:31,  8.49s/it]

torch.Size([312, 9683, 12])


  8%|▊         | 3/40 [00:26<05:11,  8.42s/it]

...

torch.Size([312, 9683, 12])


 98%|█████████▊| 39/40 [03:07<00:04,  4.24s/it]

torch.Size([312, 9683, 12])


100%|██████████| 40/40 [03:11<00:00,  4.80s/it]

torch.Size([312, 9683, 12])


In [37]:
# Shuffle and split test for mean / var test data. 
model.eval()
cost = 0
test_dataset = torch.tensor(X_test).double()
batch_size = X_test.shape[1]
hidden = model.init_hidden(batch_size = batch_size)
y_hat = model(test_dataset, hidden, batch_size=batch_size)
cost = cost + torch.mean((y_hat[0].data-Y_test)**2)
test_mse = cost.item()
print("MSE: {:.4f}".format(test_mse))

torch.Size([72, 9683, 12])
MSE: 0.3801
